In [1]:
pip install datasets nltk

In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datasets


In [ ]:
# Load XSum dataset from Hugging Face datasets library
xsum_dataset = datasets.load_dataset("xsum")

# Extract articles and summaries from the XSum dataset
train_articles = xsum_dataset["train"]["document"]
train_summaries = xsum_dataset["train"]["summary"]

test_articles = xsum_dataset["test"]["document"]
test_summaries = xsum_dataset["test"]["summary"]

# Tokenize and preprocess the text data
max_words = 10000
max_len = 200  # Adjust as needed

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_articles + train_summaries)

X_train_seq = tokenizer.texts_to_sequences(train_articles)
X_test_seq = tokenizer.texts_to_sequences(test_articles)

X_train = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Tokenize and preprocess the target summaries
Y_train_seq = tokenizer.texts_to_sequences(train_summaries)
Y_test_seq = tokenizer.texts_to_sequences(test_summaries)

Y_train = pad_sequences(Y_train_seq, maxlen=max_len, padding='post')
Y_test = pad_sequences(Y_test_seq, maxlen=max_len, padding='post')

In [ ]:
# Build the RNN model (similar to the previous example)
embedding_dim = 50
vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
# embedding_dim = 100
# vocab_size = len(tokenizer.word_index) + 1

# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.LSTM(32),
#     tf.keras.layers.Dense(vocab_size, activation='softmax')
# ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')